In [6]:
from pathlib import Path
import os
os.getcwd()

'/Users/devenderswami/GenAI/GenAI-NoteBooks'

In [7]:
API_KEY = Path("api_key.txt").read_text().strip()
INDEX_NAME = "vectordbsearch"
MODEL_NAME = "sentence-transformers/paraphrase-MiniLM-L6-v2"

In [25]:
HTML_DIR = Path(r"/Users/devenderswami/GenAI/GenAI-NoteBooks/html_documents")

In [26]:
from pinecone import Pinecone,ServerlessSpec

In [27]:
pc = Pinecone(api_key=API_KEY)

In [28]:
existing = [getattr(ix, "name", None) or ix.get("name") for ix in pc.list_indexes()]
if INDEX_NAME not in existing:
    pc.create_index(
        name=INDEX_NAME,
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


index = pc.Index(INDEX_NAME)

In [29]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(MODEL_NAME)

In [30]:
from bs4 import BeautifulSoup
for fp in HTML_DIR.glob("*.html"):
    text = BeautifulSoup(fp.read_text(encoding="utf-8"),
    "html.parser").get_text()
    vec = model.encode(text).tolist()
    index.upsert([(fp.stem, vec)]) # (id, values)

In [31]:
q = "Reward points can be redeemed for what??"
qvec = model.encode(q).tolist()

res = index.query(vector=qvec, top_k=5, include_values=False,include_metadata=False)
for m in res["matches"]:
     print(m["id"], round(m["score"], 4))


credit_card_policy 0.4069
sample_policy 0.0628
